In [1]:
# Author: Jaehun Kim
# Email: rlawogns1204@unist.ac.kr
# Affiliation: UNIST BME BCILAB
# Date: 2023-05-24
#
# This code implements a tactile information processing model using a spiking
# neural network (SNN). It simulates the processing of tactile information from
# mechanoreceptors in the skin through primary afferent fibers (PA), cuneate nucleus
# neurons (PN and IN), and ultimately, somatosensory cortex neurons. The model
# incorporates lateral inhibition and various receptive field properties to
# represent a realistic processing of touch stimuli.

# with DIGIT-sensor
# pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import scipy.io
import time
import numpy as np
from PIL import Image
import io
import datetime

from Function.ReceptiveField import *
from Function.SNNModule import *
from Function.plot_SNN import *
from Function.SNN_DIGIT import *
from Function.Stimulation_GPU import *
import cv2

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Check if GPU is available and print the CUDA version
print(f"GPU available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
# Enable GPU support if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the device to 'cpu'
device = 'cpu'
print(f"Using device: {device}")
# Start of the SNN (Spiking Neural Network) code

GPU available: True
CUDA version: 10.1
Using device: cpu


In [3]:
# Set sensor dimensions (height and width in millimeters)
sensor_h, sensor_w = 19, 16
# Set pixel dimensions (number of pixels in height and width)
pixel_h, pixel_w = 64, 48
# Set image frames per second (FPS) of tactile sensor
image_FPS = 60

In [4]:
#1st layer
############################################################################################################################################################
# Generate Slowly Adapting (SA) and Rapidly Adapting (RA) receptive fields
SA_RF, [SA_rf_height, SA_rf_width] = generate_mechanoreceptor_to_afferent_rf(kernel_w=9, kernel_h=11, step_size=5, device=device)
RA_RF, [RA_rf_height, RA_rf_width] = generate_mechanoreceptor_to_afferent_rf(kernel_w=11, kernel_h=14, step_size=4, device=device)
# Print the shape of the SA_rf variable
print("SA_rf shape:", SA_RF.shape, 'with height =',SA_rf_height, 'with width =', SA_rf_width)
print("RA_rf shape:", RA_RF.shape, 'with height =',RA_rf_height, 'with width =', RA_rf_width)
############################################################################################################################################################

#2nd layer
############################################################################################################################################################
# Define optimized receptive fields and synaptic delays
CN_PN_RF = [torch.tensor([[0, 0, 0], [0, 1, 0], [0, 0, 0]],device=device) * 4]
CN_IN_RF = [torch.tensor([[1, 1, 1], [1, 0, 1], [1, 1, 1]],device=device)]
CN_SD = [torch.tensor([[2, 2, 2], [2, 2, 2], [2, 2, 2]], device=device)]

CN_INtoPN_RF = []

# Check if the sizes of the inner tensors are different and print the index
for i, (PN, IN) in enumerate(zip(CN_PN_RF, CN_IN_RF)):
    if PN.size() != IN.size():
        raise ValueError(
            f"The inner tensors at index {i} have different sizes: {PN.size()} != {IN.size()}")

SA_CN_PN_RF, [SA_CN_PN_step_height, SA_CN_PN_step_width] = generate_weight(CN_PN_RF, pixel_h=SA_rf_height,pixel_w=SA_rf_width, step_size=1, device=device)
SA_CN_IN_RF, [SA_CN_IN_step_height, SA_CN_IN_step_width] = generate_weight(CN_IN_RF, pixel_h=SA_rf_height,pixel_w=SA_rf_width, step_size=1, device=device)
SA_CN_SD, [SA_CN_SD_step_height, SA_CN_SD_step_width]  = generate_weight(CN_SD, pixel_h=SA_rf_height,pixel_w=SA_rf_width, step_size=1, device=device)
RA_CN_PN_RF, [RA_CN_PN_step_height, RA_CN_PN_step_width] = generate_weight(CN_PN_RF, pixel_h=RA_rf_height,pixel_w=RA_rf_width, step_size=1, device=device)
RA_CN_IN_RF, [RA_CN_IN_step_height, RA_CN_IN_step_width] = generate_weight(CN_IN_RF, pixel_h=RA_rf_height,pixel_w=RA_rf_width, step_size=1, device=device)
RA_CN_SD, [RA_CN_SD_step_height, RA_CN_SD_step_width] = generate_weight(CN_SD, pixel_h=RA_rf_height, pixel_w=RA_rf_width, step_size=1, device=device)

SA_INtoPN_RF, SA_INtoPN_DN = create_weight_matrix(len(SA_CN_IN_RF), len(SA_CN_PN_RF), connection_probability=0.2, device=device)
RA_INtoPN_RF, RA_INtoPN_DN = create_weight_matrix(len(RA_CN_IN_RF), len(RA_CN_PN_RF), connection_probability=0.2, device=device)

print("SA_CN_PN_RF shape: ", SA_CN_PN_RF.shape,"SA_CN_PN_step_height:", SA_CN_PN_step_height,"SA_CN_PN_step_width:", SA_CN_PN_step_width)
print("SA_CN_IN_RF shape: ", SA_CN_IN_RF.shape,"SA_CN_IN_step_height:", SA_CN_IN_step_height,"SA_CN_IN_step_width:", SA_CN_IN_step_width)
print("RA_CN_PN_RF shape: ", RA_CN_PN_RF.shape,"RA_CN_PN_step_height:", RA_CN_PN_step_height,"RA_CN_PN_step_width:", RA_CN_PN_step_width)
print("RA_CN_IN_RF shape: ", RA_CN_IN_RF.shape,"RA_CN_IN_step_height:", RA_CN_IN_step_height,"RA_CN_IN_step_width:", RA_CN_IN_step_width)
print("SA_INtoPN_RF shape: ", SA_INtoPN_RF.shape)
print("RA_INtoPN_RF shape: ", RA_INtoPN_RF.shape)
############################################################################################################################################################

#3rd layer
############################################################################################################################################################
# 3rd integration layer
# Define optimized receptive fields and synaptic delays
CN_PN_RF_set = [torch.tensor([[0, 0, 0], [0, 0, 0], [1, 1 ,1]], device=device),torch.tensor([[0, 0, 1], [0, 0, 1], [0, 0 ,1]], device=device),
                torch.tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]], device=device),torch.tensor([[0, 0, 1], [0, 1, 0], [1, 0, 0]], device=device)]
CN_IN_RF_set = [torch.tensor([[0, 0, 0], [1, 1 ,1], [0, 0 ,0]], device=device),torch.tensor([[0, 1, 0], [0, 1 ,0], [0, 1 ,0]], device=device),
                torch.tensor([[0, 1, 0], [0, 0, 1], [0, 0, 0]], device=device)*3/2,torch.tensor([[0, 1, 0], [1, 0, 0], [0, 0, 0]], device=device)*3/2]
CN_SD_set = [torch.tensor([[2, 2, 2], [2, 2, 2], [2, 2, 2]], device=device),torch.tensor([[2, 2, 2], [2, 2, 2], [2, 2, 2]], device=device),
             torch.tensor([[2, 2, 2], [2, 2, 2], [2, 2, 2]], device=device),torch.tensor([[2, 2, 2], [2, 2, 2], [2, 2, 2]], device=device)]
rf_sizes=[(7, 4)]

CN_PN_RF_RA_set = [torch.tensor([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0],[0, 0, 0, 0, 0],[1, 1, 1 ,1, 1], [1, 1, 1 ,1, 1]], device=device)/10*3,
                   torch.tensor([[0, 0, 0, 1, 1], [0, 0, 0, 1, 1],[0, 0, 0, 1, 1],[0, 0, 0 ,1, 1], [0, 0, 0 ,1, 1]], device=device)/10*3,
                   torch.tensor([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]], device=device)/5*3,
                   torch.tensor([[0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0]], device=device)/5*3]

CN_IN_RF_RA_set = [torch.tensor([[0, 0, 0, 0, 0], [1, 1, 1 ,1, 1], [1, 1, 1 ,1, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], device=device)/10*3,
                   torch.tensor([[0, 1, 1, 0, 0], [0, 1, 1, 0, 0],[0, 1, 1, 0, 0],[0, 1, 1, 0, 0], [0, 1, 1, 0, 0]], device=device)/10*3,
                   torch.tensor([[0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]], device=device)/5*3,
                   torch.tensor([[0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0]], device=device)/5*3,]

CN_SD__RA_set = [torch.tensor([[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2]], device=device),
                 torch.tensor([[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2]], device=device),
                 torch.tensor([[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2]], device=device),
                 torch.tensor([[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2],[2, 2, 2 ,2, 2]], device=device)]


# CN_PN_RF, CN_PN_DN = create_weight_matrix(len(SA_CN_PN_RF)+len(RA_CN_PN_RF),num_output_neuron,connection_probability = 0.2, device = device)
CN_PN_SA_RF, [CN_PN_SA_RF_step_height, CN_PN_SA_RF_step_width] = generate_weight(CN_PN_RF_set, pixel_h=SA_CN_PN_step_height, pixel_w=SA_CN_PN_step_width, step_size=1, device=device)
CN_IN_SA_RF, [CN_IN_SA_RF_step_height, CN_IN_SA_RF_step_width] = generate_weight(CN_IN_RF_set, pixel_h=SA_CN_PN_step_height, pixel_w=SA_CN_PN_step_width, step_size=1, device=device)
CN_SA_SD, [CN_SA_SD_step_height, CN_SA_SD_step_width] = generate_weight(
    CN_SD_set, pixel_h=SA_CN_PN_step_height, pixel_w=SA_CN_PN_step_width, step_size=1, device=device)

CN_PN_RA_RF, [CN_PN_RA_RF_step_height, CN_PN_RA_RF_step_width] = generate_weight(CN_PN_RF_RA_set, pixel_h=RA_CN_PN_step_height, pixel_w=RA_CN_PN_step_width, step_size=1, device=device)
CN_IN_RA_RF, [CN_IN_RA_RF_step_height, CN_IN_RA_RF_step_width] = generate_weight(CN_IN_RF_RA_set, pixel_h=RA_CN_PN_step_height, pixel_w=RA_CN_PN_step_width, step_size=1, device=device)
CN_RA_SD, [CN_RA_SD_step_height, CN_RA_SD_step_width] = generate_weight(
    CN_SD__RA_set, pixel_h=RA_CN_PN_step_height, pixel_w=RA_CN_PN_step_width, step_size=1, device=device)

CN_INtoPN_RF, CN_INtoPN_DN = create_weight_matrix(len(CN_IN_SA_RF), len(CN_PN_SA_RF), connection_probability=0.2, device=device)

print("CN_PN_SA_RF shape: ", CN_PN_SA_RF.shape, "CN_PN_SA_RF_step_height:", CN_PN_SA_RF_step_height, "CN_PN_SA_RF_step_width:", CN_PN_SA_RF_step_width)
print("CN_IN_SA_RF shape: ", CN_IN_SA_RF.shape, "CN_IN_SA_RF_step_height:", CN_IN_SA_RF_step_height, "CN_IN_SA_RF_step_width:", CN_IN_SA_RF_step_width)
print("CN_PN_RA_RF shape: ", CN_PN_RA_RF.shape, "CN_PN_RA_RF_step_height:", CN_PN_RA_RF_step_height, "CN_PN_RA_RF_step_width:", CN_PN_RA_RF_step_width)
print("CN_IN_RA_RF shape: ", CN_IN_RA_RF.shape, "CN_IN_RA_RF_step_height:", CN_IN_RA_RF_step_height, "CN_IN_RA_RF_step_width:", CN_IN_RA_RF_step_width)
print("CN_INtoPN_RF shape: ", CN_INtoPN_RF.shape)
############################################################################################################################################################

SA_rf shape: torch.Size([88, 3072]) with height = 11 with width = 8
RA_rf shape: torch.Size([130, 3072]) with height = 13 with width = 10
SA_CN_PN_RF shape:  torch.Size([54, 88]) SA_CN_PN_step_height: 9 SA_CN_PN_step_width: 6
SA_CN_IN_RF shape:  torch.Size([54, 88]) SA_CN_IN_step_height: 9 SA_CN_IN_step_width: 6
RA_CN_PN_RF shape:  torch.Size([88, 130]) RA_CN_PN_step_height: 11 RA_CN_PN_step_width: 8
RA_CN_IN_RF shape:  torch.Size([88, 130]) RA_CN_IN_step_height: 11 RA_CN_IN_step_width: 8
SA_INtoPN_RF shape:  torch.Size([54, 54])
RA_INtoPN_RF shape:  torch.Size([88, 88])
CN_PN_SA_RF shape:  torch.Size([112, 54]) CN_PN_SA_RF_step_height: 7 CN_PN_SA_RF_step_width: 4
CN_IN_SA_RF shape:  torch.Size([112, 54]) CN_IN_SA_RF_step_height: 7 CN_IN_SA_RF_step_width: 4
CN_PN_RA_RF shape:  torch.Size([112, 88]) CN_PN_RA_RF_step_height: 7 CN_PN_RA_RF_step_width: 4
CN_IN_RA_RF shape:  torch.Size([112, 88]) CN_IN_RA_RF_step_height: 7 CN_IN_RA_RF_step_width: 4
CN_INtoPN_RF shape:  torch.Size([112, 112]

In [5]:
noise_std_val = 4

# Create Izhikevich layers
SA_layer = IzhikevichLayer(0.02, 0.2, -65, 8, len(SA_RF),v_thres=30, a_decay=1.02, noise_std = noise_std_val, device=device)
SA_CN_IN_layer = IzhikevichLayer(0.1, 0.2, -65, 6, len(SA_CN_IN_RF),v_thres=30, a_decay=1, noise_std = noise_std_val, device=device)
SA_CN_PN_layer = IzhikevichLayer(0.1, 0.2, -65, 6, len(SA_CN_PN_RF),v_thres=30, a_decay=1, noise_std = noise_std_val, device=device)

RA_layer = IzhikevichLayer(0.02, 0.2, -65, 8, len(RA_RF),v_thres=30, a_decay=1, noise_std = noise_std_val,device=device)
RA_CN_IN_layer = IzhikevichLayer(0.1, 0.2, -65, 6, len(RA_CN_IN_RF),v_thres=30, a_decay=1, noise_std=noise_std_val, device=device)
RA_CN_PN_layer = IzhikevichLayer(0.1, 0.2, -65, 6, len(RA_CN_PN_RF),v_thres=30, a_decay=1,noise_std = noise_std_val, device=device)

SA_layers = [SA_layer, SA_CN_IN_layer, SA_CN_PN_layer]
RA_layers = [RA_layer, RA_CN_IN_layer, RA_CN_PN_layer]

SA_synapse = Synapse(SA_RF.to(device), device=device)
SA_CN_IN_synapse = Synapse(SA_CN_IN_RF.to(device), delays=SA_CN_SD, device=device)
SA_CN_PN_synapse = Synapse(SA_CN_PN_RF.to(device), delays=SA_CN_SD, device=device)
SA_CN_INtoPN_synapse = Synapse(SA_INtoPN_RF.to(device), delays = SA_INtoPN_DN, tau_psp = 10, device = device)

RA_synapse = Synapse(RA_RF.to(device), device=device)
RA_CN_IN_synapse = Synapse(RA_CN_IN_RF.to(device), delays=RA_CN_SD, device=device)
RA_CN_PN_synapse = Synapse(RA_CN_PN_RF.to(device), delays=RA_CN_SD, device=device)
RA_CN_INtoPN_synapse = Synapse(RA_INtoPN_RF.to(device), delays = RA_INtoPN_DN, tau_psp = 10, device = device)

SA_synapses = [SA_synapse, SA_CN_IN_synapse, SA_CN_PN_synapse, SA_CN_INtoPN_synapse]
RA_synapses = [RA_synapse, RA_CN_IN_synapse, RA_CN_PN_synapse, RA_CN_INtoPN_synapse]

# 3rd layer 
CN_IN_layer = IzhikevichLayer(0.1, 0.2, -65, 6, len(CN_IN_SA_RF), v_thres=30, a_decay=1, noise_std = noise_std_val, device=device)
CN_PN_layer = IzhikevichLayer(0.1, 0.2, -65, 6, len(CN_PN_SA_RF),v_thres=30, a_decay=1, noise_std = noise_std_val, device=device)

CN_IN_SA_synapse = Synapse(CN_IN_SA_RF.to(device), delays=CN_SA_SD, device=device)
CN_PN_SA_synapse = Synapse(CN_PN_SA_RF.to(device), delays=CN_SA_SD, device=device)
CN_IN_RA_synapse = Synapse(CN_IN_RA_RF.to(device), delays=CN_RA_SD, device=device)
CN_PN_RA_synapse = Synapse(CN_PN_RA_RF.to(device), delays=CN_RA_SD, device=device)

CN_INtoPN_synapse = Synapse(CN_INtoPN_RF.to(device), delays = CN_INtoPN_DN, tau_psp = 10, device = device)

CN_layers = [CN_IN_layer, CN_PN_layer]
CN_synapses = [CN_IN_SA_synapse, CN_PN_SA_synapse, CN_IN_RA_synapse, CN_PN_RA_synapse, CN_INtoPN_synapse]



In [76]:
print(datetime.datetime.now())

S = SNNModel([SA_layers, RA_layers, CN_layers], [SA_synapses, RA_synapses, CN_synapses], rf_sizes=rf_sizes, device = device)

S.feedforward()

2023-06-01 17:48:18.723390
start feedforward
Tactile sensor is open.
FPS: 60.0
procesing start
0.05900287628173828
processing end
Tactile sensor is closed.


In [ ]:
import cv2
import numpy as np

def get_sensor_input(frame):
    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Flip the frame vertically and rotate it counterclockwise by 90 degrees
    flipped_frame = cv2.flip(gray_frame, 0)
    rotated_frame = cv2.rotate(flipped_frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    
    # Resize the frame to desired dimensions
    resized_frame = cv2.resize(rotated_frame, (48, 64))

    # Apply Sobel edge detection
    sobelx = cv2.Sobel(resized_frame, cv2.CV_64F, 1, 0, ksize=5)  # x-axis sobel edge detection
    sobely = cv2.Sobel(resized_frame, cv2.CV_64F, 0, 1, ksize=5)  # y-axis sobel edge detection
    sobel_frame = np.hypot(sobelx, sobely)  # combining both edges

    # Normalize the result to 0-255
    sobel_frame = sobel_frame / sobel_frame.max() * 255

    return sobel_frame

In [33]:
def set_initial_sensor_settings(sensor):
    desired_width = 320
    desired_height = 240
    sensor.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
    sensor.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

    sensor.set(cv2.CAP_PROP_FPS, 60)
    fps = sensor.get(cv2.CAP_PROP_FPS)
    print(f"FPS: {fps}")


In [46]:
def get_sensor_input(sensor, plot_image  = True):
    ret, frame = sensor.read() 

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flipped_frame = cv2.flip(gray_frame, 0)
    rotated_frame = cv2.rotate(flipped_frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
    resized_frame = cv2.resize(rotated_frame, (48, 64))

    sobelx = cv2.Sobel(resized_frame, cv2.CV_64F, 1, 0, ksize=3)  # x-axis sobel edge detection # combining both edges  # x-axis sobel edge detection

    if plot_image == True:
        # cv2.imshow('frame', resized_frame)
        cv2.imshow('sobel', sobelx)

    return resized_frame

In [47]:
import time
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

set_initial_sensor_settings(cap)

frames = []  # Frame을 저장할 리스트
frame_count = 0  # 캡처한 Frame 수
total_time = 0  # 측정 시간의 합계

while True: 
    startT = time.time()

    frame = get_sensor_input(cap, True)

    frames.append(frame)  # 변경된 Frame을 리스트에 추가
    frame_count += 1  # Frame 수 증가
    
    endT = time.time()
    total_time += (endT - startT)  # 측정 시간 누적
    # 10ms 기다리고 다음 프레임으로 전환, Esc누르면 while 강제 종료
    if cv2.waitKey(1) == 27 or frame_count == 600:
        break

cap.release() 
cv2.destroyAllWindows()

avg_time = total_time / frame_count if frame_count else 0  # 평균 실행 시간 계산
print(f"Average Time per Frame: {avg_time} sec")  # 평균 실행 시간 출력


FPS: 60.0
Average Time per Frame: 0.009654074907302856 sec
